# Instructions

The Stanford40_JPEGImages and Stanford40_XMLAnnotations folders needs to be in the embedded_project.

In the Stanford40_JPEGImages, add train.text and test.txt into that folder from Stanford40_ImageSplits\ImageSplits

In [2]:
import os, cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import shutil

In [3]:
imgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\JPEGImages'
parentImgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages'
xmlFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'
directory = "CImages"


In [4]:
def extractBBox():
    os.chdir(xmlFileDir)
    imgBBoxes = []
    xmlFileNames = [f for f in listdir(xmlFileDir) if isfile(join(xmlFileDir, f))]
    #print(xmlFileNames[:10])
    actionList = []
    for file in xmlFileNames:
        root = ET.parse(file).getroot()
        filename = root.find('filename').text
        xmin = int(root.find('object/bndbox/xmin').text)
        ymin = int(root.find('object/bndbox/ymin').text)
        xmax = int(root.find('object/bndbox/xmax').text)
        ymax = int(root.find('object/bndbox/ymax').text)
        action = root.find('object/action').text
        actionList.append(action)
        imgBBoxes.append([filename, xmin, ymin, xmax, ymax])
    print("Done")
    return imgBBoxes, actionList
BBoxes, actionList = extractBBox()

Done


In [ ]:
def saveClassOrder(actionList):
    (classUnique,counts) = np.unique(actionList, return_counts=True)
    np.save(r'C:\Users\WillyB\Documents\GitHub\embedded_project\code/classOrder.npy', np.array(classUnique))
    print("classOrder.npy is created")
#saveClassOrder()

In [9]:
def create_classFolders():
    (classUnique,counts) = np.unique(actionList, return_counts=True)
    os.chdir(parentImgFileDir)
    actionImagesPath = os.path.join(parentImgFileDir, "ActionImages")
    if os.path.exists(actionImagesPath):
        print("it exist")
    else:
        print("does not exist, create folder")
        os.mkdir(actionImagesPath)
    for actionName in classUnique:
        path = os.path.join(actionImagesPath, actionName)
        print(path+'/')
        if os.path.exists(path):
            print("it exist")
        else:
            print("does not exist, create folder")
            os.mkdir(path)


create_classFolders()

does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\applauding/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\blowing_bubbles/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\brushing_teeth/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\cleaning_the_floor/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\climbing/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\cooking/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages\cutting_trees/
does not exist, create folder
C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGI

array(['applauding', 'blowing_bubbles', 'brushing_teeth',
       'cleaning_the_floor', 'climbing', 'cooking', 'cutting_trees',
       'cutting_vegetables', 'drinking', 'feeding_a_horse', 'fishing',
       'fixing_a_bike', 'fixing_a_car', 'gardening',
       'holding_an_umbrella', 'jumping', 'looking_through_a_microscope',
       'looking_through_a_telescope', 'phoning', 'playing_guitar',
       'playing_violin', 'pouring_liquid', 'pushing_a_cart', 'reading',
       'riding_a_bike', 'riding_a_horse', 'rowing_a_boat', 'running',
       'shooting_an_arrow', 'smoking', 'taking_photos', 'texting_message',
       'throwing_frisby', 'using_a_computer', 'walking_the_dog',
       'washing_dishes', 'watching_TV', 'waving_hands',
       'writing_on_a_board', 'writing_on_a_book'], dtype='<U28')

In [4]:
def cropImages(BBoxes):
    #make cropped images folder
    path = os.path.join(parentImgFileDir, directory)
    print(path+'/')
    if os.path.exists(path):
        print("it exist")
    else:
        print("does not exist, create folder")
        os.mkdir(path)
    
    os.chdir(imgFileDir)
    count = 0
    for image in BBoxes:
        #print("cropping: ", image[0])
        print(str(count)+"/"+str(len(BBoxes)), end='\r')
        count +=1
        im =Image.open(image[0])
        xmin = image[1]
        ymin = image[2]
        xmax = image[3]
        ymax = image[4]
        cImg = im.crop((xmin, ymin, xmax,ymax))
        cImg.save(path+'/'+image[0], 'JPEG')
    print()
    print("Done")
cropImages(BBoxes)

C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\CImages/
does not exist, create folder
9531/9532
Done


In [11]:
def moveActionImages():
    actionImagesPath = os.path.join(parentImgFileDir, "ActionImages")
    croppedImgPath = os.path.join(parentImgFileDir, directory)
    os.chdir(croppedImgPath)

    for x in range(len(BBoxes)):
        shutil.copy(BBoxes[x][0], os.path.join(actionImagesPath,actionList[x]))
        print(str(x+1)+"/"+str(len(BBoxes)), end='\r')
        
moveActionImages()


In [5]:
def moveImgTrainTest():
    #make cropped images folder
    croppedImgPath = os.path.join(parentImgFileDir, directory)
    os.chdir(parentImgFileDir)

    #create Test and Train folders
    train_path = os.path.join(parentImgFileDir, "Train")
    test_path = os.path.join(parentImgFileDir, "Test")
    if os.path.exists(train_path):
        print("Train folder already exists")
    else:
        print("Train folder does not exist, create folder")
        os.mkdir(train_path)
    if os.path.exists(test_path):
        print("Test folder already exists")
    else:
        print("Test folder does not exist, create folder")
        os.mkdir(test_path)
    trainlist = [line.rstrip('\n') for line in open('train.txt', 'r')]
    #testlist = open('test.txt', 'r').readlines()
    testlist = [line.rstrip('\n') for line in open('test.txt', 'r')]
    
    #Copy Test and Train images to the correct folder
    os.chdir(croppedImgPath)
    test_count = 0
    train_count = 0
    for img in trainlist:
        #print(str(test_count)+'/'+str(len(trainlist)), end='\r')
        shutil.copy(img, train_path)
        train_count += 1
    print()
    print("moved all train images", train_count)
    
    for img in testlist:
        shutil.copy(img, test_path)
        #print(str(train_count)+'/'+str(len(testlist)), end='\r')
        test_count += 1
    print()
    print("moved all test images", test_count)


moveImgTrainTest()


Train folder does not exist, create folder
Test folder does not exist, create folder

moved all train images 4000

moved all test images 5532
